# Imports

In [1]:
from _util import * 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from _helpers_parsing import driver_setup
from concurrent.futures import ThreadPoolExecutor

In [2]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime
import random
from collections import Counter
import os
from tqdm import tqdm
tqdm.pandas()
random.seed(42)

# Functions

In [3]:
def get_jobdescription_eures(ads, driver, filename):
    results = []
    dead_links = []
    #ads = [ad for ad in ads if ad["url"] not in urls_parsed]
    for ad in tqdm(ads):
        driver.get(ad["url"]+"?jvDisplayLanguage=de&lang=de")
        try:
            #if error message available, cancel parsing and add the link to the dead link list
            WebDriverWait(driver,3).until(EC.presence_of_element_located((By.ID, "error-message-jv-detail")))
            dead_links.append(ad)
            with open(filename+"_deadlinks", 'w',encoding= "utf-8") as fp:
                json.dump(dead_links, fp, indent = 2, ensure_ascii=False)
            continue
        except:
            pass
        try:
            WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.ID, "jv-details-job-description")))
            WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.ID, "jv-job-categories-codes")))
            time.sleep(0.5)
            soup = BeautifulSoup(driver.page_source, "html.parser")
        except:
            continue

        try:
            jobtitle = soup.find("h1").text
        except:
            jobtitle = None
        try:
            desc_content = (soup.find(id="jv-details-job-description").contents)
            description = ""
            for i in range(len(desc_content)):
                if desc_content[i].text != "":
                    description += desc_content[i].text.replace("\xa0","")
                if i+1 < len(desc_content):
                    if str(desc_content[i]) == "<br/>" and str(desc_content[i+1]) == "<br/>":
                        description+= " \n "
        except:
            description = None
        # try:
        #     reqs = soup.find(id="jv-details-job-requirements-section").text
        # except:
        #     reqs = None
        #try:
        jobs_esco_container = soup.find(id="jv-job-categories-codes")
        jobs_esco_list = jobs_esco_container.find_all(class_="ecl-u-ml-2xs ng-star-inserted")
        #print(jobs_esco_list[0])
        ESCO_JOBS = [job.text.replace("  -","").strip() for job in jobs_esco_list]
        #except:
            #ESCO_JOBS = None
        ad.update({"parsed_title":jobtitle, "ESCOJOB":ESCO_JOBS, "description": description})
        results.append(ad)
        if len(results)%50 == 0:
            with open(filename, 'w',encoding= "utf-8") as fp:
                json.dump(results, fp, indent = 2, ensure_ascii=False)
    with open(filename, 'w',encoding= "utf-8") as fp:
        json.dump(results, fp, indent = 2, ensure_ascii=False)
    driver.quit()
    return results

In [4]:
def parse_multithreading_eures(df, urls_parsed, headless=True):
    drivers = [driver_setup(headless) for _ in range(4)]
    currently = "".join([c for c in str(datetime.now()).split('.')[0] if c.isdigit()])
    filenames = [f"../00_data/EURES/{currently}_eres_ads_{i}.json" for i in range(1,5)]
    print("Sorting out parsed ads.")
    urllist = df[~df["url"].isin(urls_parsed)].to_dict("records")
    print(f"Parsing {len(urllist)} ads.")
    #urllist = [x for x in tqdm(urllist) if x["url"] not in set(urls_parsed)]
    chunks = np.array_split(urllist, 4)
    with ThreadPoolExecutor(max_workers=4) as executor:  
        bucket = executor.map(get_jobdescription_eures, chunks, drivers, filenames)
        results = [item for block in bucket for item in block]
    write_json(f"00_data/03_output/eures_results/{currently}_eres_ads_total.json",results)
    return results

In [5]:
# def choose_random_ads(df, k):
#     random_choices = []
#     ids_unique = df["esco_id"].unique()
#     for id in tqdm(ids_unique):
#         filtered_df = df[df["esco_id"] == id]
#         if len(filtered_df) <= k:
#             random_choices += filtered_df.to_dict("records")
#         else:
#             random_choices += random.sample(filtered_df.to_dict("records"), k)
#     return random_choices

# Main

# load overview

In [6]:
import pandas as pd

In [7]:
overview = pd.DataFrame(load_json("../00_data/EURES/eures_overview_total.json"))
print(len(overview))
overview = overview.drop_duplicates(["title", "url"])
print(len(overview))
overview = overview[~overview["title"].isnull()]
print(len(overview))
overview.head(2)

212883
161179
161178


,searched_esco_job,title,url,esco_jobs,publication_date
0,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,Sachbearbeiter landwirtschaftlicher Gewässers...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,[Wasserqualitätsanalytiker/Wasserqualitätsanal...,24.03.2023
1,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,Sachbearbeiter landwirtschaftlicher Gewässers...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,[Wasserqualitätsanalytiker/Wasserqualitätsanal...,22.03.2023


## add esco ids

In [8]:
esco_lookup_dict = load_json("../00_data/ESCO/esco_lookup.json")

In [9]:
# if a job advertisement has multiple esco jobs annotated --> extend to multiple rows
# makes the data set larger with unique ads in [text & esco id]
def extend_jobs(ad):
    extended = []
    for job in ad["esco_jobs"]:
        job_ext = dict(ad)
        job_ext["esco_job"] = job
        try:
            id = esco_lookup_dict[job.lower()]
            job_ext["esco_id"] = id
        except:
            job_ext["esco_id"] = None
            continue
        if "." in id:
            extended.append(job_ext)
    return extended

In [10]:
# Dataset is extended to full lenght
ads_extended = pd.DataFrame(flatten_list(overview.progress_apply(extend_jobs, axis=1)))
print(len(ads_extended))
ads_extended = ads_extended[~ads_extended["esco_id"].isnull()]
print(len(ads_extended))

  0%|          | 0/161178 [00:00<?, ?it/s]

100%|██████████| 161178/161178 [00:11<00:00, 14051.75it/s]


266177
266177


In [11]:
unique_ids_unparsed = list(ads_extended["esco_id"].unique())
len(unique_ids_unparsed)

1703

## load parsed ads

In [12]:
parsed_jobads = pd.DataFrame(load_json("../00_data/EURES/parsed_ads_final.json"))
len(parsed_jobads)
desc_lookup = parsed_jobads[["url","description"]].drop_duplicates(["url", "description"])
len(desc_lookup)

86124

In [13]:
parsed_jobads.head()

,searched_esco_job,title,url,esco_jobs,publication_date,esco_job,esco_id,description,has_alpha,length,count
0,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,Sachbearbeiter landwirtschaftlicher Gewässers...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,[Wasserqualitätsanalytiker/Wasserqualitätsanal...,24.03.2023,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,2133.12,Stellenangebotsbeschreibung: Im Landkreis Spre...,True,752.0,2
1,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,Sachbearbeiter landwirtschaftlicher Gewässers...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,[Wasserqualitätsanalytiker/Wasserqualitätsanal...,22.03.2023,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,2133.12,Stellenangebotsbeschreibung: Im Landkreis Spre...,True,752.0,2
2,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,Sachbearbeiter landwirtschaftlicher Gewässers...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,[Wasserqualitätsanalytiker/Wasserqualitätsanal...,24.03.2023,Büroassistent/Büroassistentin,4110.1,Stellenangebotsbeschreibung: Im Landkreis Spre...,True,752.0,935
3,Wasserqualitätsanalytiker/Wasserqualitätsanaly...,Sachbearbeiter landwirtschaftlicher Gewässers...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,[Wasserqualitätsanalytiker/Wasserqualitätsanal...,22.03.2023,Büroassistent/Büroassistentin,4110.1,Stellenangebotsbeschreibung: Im Landkreis Spre...,True,752.0,935
4,Chemieingenieur/Chemieingenieurin,Sachbearbeitung Grundwasser (Ingenieure/Ingen...,https://ec.europa.eu/eures/portal/jv-se/jv-det...,"[Büroassistent/Büroassistentin, Bauingenieur/B...",24.03.2023,Büroassistent/Büroassistentin,4110.1,Stellenangebotsbeschreibung: Sachbearbeitung G...,True,6031.0,935


#### quality check of descriptions

In [14]:
def alphabetic_char(description):
    for c in description:
        if c.isalpha():
            return True
    return False

In [15]:
desc_lookup["has_alpha"] = desc_lookup["description"].apply(alphabetic_char)
desc_lookup["length"] = desc_lookup["description"].apply(len)
desc_lookup.sort_values(by="length").iloc[5]["description"]

'Stellenangebotsbeschreibung: bedienen von gästen und empfelungen miteilen'

#### merging overview with already parsed ads

In [16]:
parsed_jobads = pd.merge(ads_extended,desc_lookup, on="url",how="left")
print(len(parsed_jobads))
# parsing errors in some descriptions
parsed_jobads = parsed_jobads[~parsed_jobads["description"].isna()]
print(len(parsed_jobads))
parsed_jobads = parsed_jobads[~parsed_jobads["esco_id"].isna()]
print(len(parsed_jobads))
unique_ids_parsed = list(parsed_jobads["esco_id"].unique())
print(f"({len(unique_ids_parsed)})")

266252
152069
152069
(1700)


In [17]:
# check whether there are still job ads from esco jobs in the overview, which are missing in the final
# dataset 
if len(unique_ids_unparsed) != len(unique_ids_parsed):
    print(f"{len(unique_ids_unparsed)}/{len(unique_ids_parsed)}")
    set_missing_ids = set(unique_ids_unparsed)-set(unique_ids_parsed)
    missing_ids = []
    for id in set_missing_ids:
        missing_ids.append({"esco_id":id, "count":0, "need":100})
    missing_ids =pd.DataFrame(missing_ids)
missing_ids

1703/1700


,esco_id,count,need
0,3141.2.4,0,100
1,1411.2,0,100
2,7223.17,0,100


In [18]:
# count numbers of esco_ids, merge on overview
counts = pd.DataFrame(parsed_jobads["esco_id"].value_counts()).reset_index()
counts.columns = ["esco_id","count"]
parsed_jobads = pd.merge(parsed_jobads, counts, on="esco_id")

####  create overview over jobads which are needed

In [19]:
in_need = counts[counts["count"]<100].copy()
in_need["need"] = 100-in_need["count"]
print(len(in_need))
in_need = pd.concat([in_need, missing_ids])
print(len(in_need))

1015
1018


In [21]:
# create set of urls, which have already been parsed
parsed_urls = (set(parsed_jobads["url"]))
deadlinks = set(load_json("../00_data/EURES/deadlinks_final.json"))
len(deadlinks)

1642

#### filter overview to unparsed urls

In [22]:
# from the unparsed overview, filter out parsed urls and dead urls
print(len(ads_extended))
unparsed_url_df = ads_extended[~ads_extended["url"].isin(parsed_urls)]
print(len(unparsed_url_df))
unparsed_url_df = unparsed_url_df[~unparsed_url_df["url"].isin(deadlinks)]
print(len(unparsed_url_df))

266177
114183
111610


#### create df of urls to parse to fill needs

In [23]:
# for every esco_id in need, filter the unparsed_url_df and
# if possible, get as many urls as needed
jobads_to_parse = []
for need in tqdm(in_need.to_dict("records")):
    filtered_need = unparsed_url_df[unparsed_url_df["esco_id"]==need["esco_id"]].iloc[0:need["need"]].to_dict("records")
    if len(filtered_need) > 0:
        jobads_to_parse += filtered_need

100%|██████████| 1018/1018 [00:13<00:00, 73.55it/s]


In [ ]:
to_parse_df = pd.DataFrame(jobads_to_parse)
to_parse_df

In [ ]:
#results = parse_multithreading_eres(random_choices_list,list(parsed_ads["url"]), headless=False)
results = parse_multithreading_eres(to_parse_df,[], headless=False)

# Results 

In [27]:
parsed_ads = (load_json("../00_data/EURES/parsed_ads_final.json"))
len(parsed_ads)

152071